## 1 - Assignment1 Neural Network model



### 1.1 Packages####

In [100]:
import numpy as np
import h5py
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import backend as K
import tensorflow as tf
from keras.layers.normalization import BatchNormalization

In [69]:
def vectorized_label(label):
    tmp = np.zeros((10, 1))
    tmp[label] = 1.0
    return tmp

### 1.2 Dataset ####
Show the first 10 instances and labels.

In [70]:
with h5py.File('train_128.h5','r') as H:
    data = np.copy(H['data'])
print("The first 10 instances: ")
print(data[:2],"\n")
with h5py.File('train_label.h5','r') as H:
    label = np.copy(H['label'])
print("The first 10 instances: ",label[:2],"\n")

print("The shape of instance is: ",data.shape)
print("The shape of label is: ",label.shape)
labels = np.array([vectorized_label(tmp) for tmp in label])
#print(labels[:10])
#data = np.array([np.reshape(tmp, (128, 1)) for tmp in data])





The first 10 instances: 
[[ -1.26502938e+02   1.63243234e+03  -1.20922145e+03   2.48416923e+02
   -6.02124110e-01  -4.04695491e+02  -9.31297310e+01   2.01038160e+02
   -3.29514187e+01  -2.77401504e+01   1.50024716e+02  -6.64644771e+01
    5.27323547e+01  -1.34801560e+02   1.02081727e+02  -5.85367859e+01
   -1.18862042e+02  -1.96611265e+02   1.87822261e+02  -4.35857526e+02
   -1.33405657e+02   1.71629407e+02   1.11454164e+02  -2.66110366e+01
   -2.06630998e+02   6.95156022e+01  -1.03421989e+02  -1.16215683e+02
   -1.16454278e+02   9.40007318e+01  -9.20309210e+01  -3.16710960e+02
   -6.25421850e+01  -7.21027152e+01  -3.22370101e+01  -5.43615649e+01
    5.58975643e+01  -1.36786583e+02   3.03174619e+01  -1.33945222e+02
   -6.15853235e+01  -7.22445118e+01   8.01202173e+01  -1.05298042e+02
   -1.13903920e+02  -1.50105891e+02   1.41943854e+02  -4.07795009e+01
   -6.84617171e+01   1.11272701e+01  -8.97342715e+01  -3.37421200e+01
   -5.39188539e+01  -7.54320096e+01  -7.08386988e+00   1.14355965

### 1.3 - Data preprocessing

In [71]:
# PCA
temp_training_set = (data-data.mean(0))

cov = temp_training_set.T.dot(temp_training_set)/temp_training_set.shape[0]
print('Starting.....PCA')
U,s,V = np.linalg.svd(cov)
#conponent pick
print('Considering...K of conponents')
valueTotal = np.sum(s)
tempTotal = 0;
k0=k1=k2=k3=k4=k5=k6=k7=k8=0
for i,v in enumerate(s):
    tempTotal +=v
    if(tempTotal/valueTotal>0.4):
        if(k0==0):
            k0=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k0+1))
    if(tempTotal/valueTotal>0.5):
        if(k1==0):
            k1=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k1+1))
    if(tempTotal/valueTotal>0.6):
        if(k2==0):
            k2=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k2+1))
    if(tempTotal/valueTotal>0.7):
        if(k3==0):
            k3=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k3+1))
    if(tempTotal/valueTotal>0.8):
        if(k4==0):
            k4=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k4+1))
    if(tempTotal/valueTotal>0.85):
        if(k5==0):
            k5=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k5+1))
    if(tempTotal/valueTotal>0.9):
        if(k6==0):
            k6=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k6+1))
    if(tempTotal/valueTotal>0.95):
        if(k7==0):
            k7=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k7+1))
    if(tempTotal/valueTotal>0.99):
        if(k8==0):
            k8=i
            print('{} % variance retained in {} dimensions'.format(tempTotal/valueTotal, k8+1))
        else:
            break
# we choose 95% remained
U_reduced = U[:, : k8+1] #as pyhon will not include the last one so add 1 
#new_traning_set = U_reduced.T.dot(training_setT)
new_traning_set =  temp_training_set.dot(U_reduced)

train_data = np.squeeze(new_traning_set[:48000])
train_label = np.squeeze(labels[:48000])
#train_label = np.expand_dims(train_label,axis=1)

print(train_data.shape,train_label.shape)

test_data = np.squeeze(new_traning_set[48000:])
test_label = np.squeeze(labels[48000:])
#test_label = np.expand_dims(test_label,axis=1)
print(test_data.shape,test_label.shape)

# mean normalization
train_data = (train_data-train_data.mean(0))
train_data = train_data/np.std(train_data,axis=0)

test_data = (test_data-test_data.mean(0))
test_data = test_data/np.std(test_data,axis=0)


Starting.....PCA
Considering...K of conponents
0.5042902733264708 % variance retained in 2 dimensions
0.5042902733264708 % variance retained in 2 dimensions
0.6225819057861219 % variance retained in 4 dimensions
0.7013422310272825 % variance retained in 6 dimensions
0.8046155203256028 % variance retained in 13 dimensions
0.8507497292328071 % variance retained in 21 dimensions
0.9010996004703893 % variance retained in 36 dimensions
0.9509384357849695 % variance retained in 65 dimensions
0.9904606017805714 % variance retained in 111 dimensions
(48000, 111) (48000, 10)
(12000, 111) (12000, 10)


### 1.4 - Defining the neural network structure ####

In [90]:
class SGDWithWeightnorm(SGD):
    def get_updates(self, params, loss):
        grads = self.get_gradients(loss, params)
        self.updates = []

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * self.iterations))
            self.updates .append(K.update_add(self.iterations, 1))

        # momentum
        shapes = [K.get_variable_shape(p) for p in params]
        moments = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + moments
        for p, g, m in zip(params, grads, moments):

            # if a weight tensor (len > 1) use weight normalized parameterization
            ps = K.get_variable_shape(p)
            if len(ps) > 1:

                # get weight normalization parameters
                V, V_norm, V_scaler, g_param, grad_g, grad_V = get_weightnorm_params_and_grads(p, g)

                # momentum container for the 'g' parameter
                V_scaler_shape = K.get_variable_shape(V_scaler)
                m_g = K.zeros(V_scaler_shape)

                # update g parameters
                v_g = self.momentum * m_g - lr * grad_g  # velocity
                self.updates.append(K.update(m_g, v_g))
                if self.nesterov:
                    new_g_param = g_param + self.momentum * v_g - lr * grad_g
                else:
                    new_g_param = g_param + v_g

                # update V parameters
                v_v = self.momentum * m - lr * grad_V  # velocity
                self.updates.append(K.update(m, v_v))
                if self.nesterov:
                    new_V_param = V + self.momentum * v_v - lr * grad_V
                else:
                    new_V_param = V + v_v

                # wn param updates --> W updates
                add_weightnorm_param_updates(self.updates, new_V_param, new_g_param, p, V_scaler)

            else: # normal SGD with momentum
                v = self.momentum * m - lr * g  # velocity
                self.updates.append(K.update(m, v))

                if self.nesterov:
                    new_p = p + self.momentum * v - lr * g
                else:
                    new_p = p + v

                self.updates.append(K.update(p, new_p))
        return self.updates
def get_weightnorm_params_and_grads(p, g):
    ps = K.get_variable_shape(p)

    # construct weight scaler: V_scaler = g/||V||
    V_scaler_shape = (ps[-1],)  # assumes we're using tensorflow!
    V_scaler = K.ones(V_scaler_shape)  # init to ones, so effective parameters don't change

    # get V parameters = ||V||/g * W
    norm_axes = [i for i in range(len(ps) - 1)]
    V = p / tf.reshape(V_scaler, [1] * len(norm_axes) + [-1])

    # split V_scaler into ||V|| and g parameters
    V_norm = tf.sqrt(tf.reduce_sum(tf.square(V), norm_axes))
    g_param = V_scaler * V_norm

    # get grad in V,g parameters
    grad_g = tf.reduce_sum(g * V, norm_axes) / V_norm
    grad_V = tf.reshape(V_scaler, [1] * len(norm_axes) + [-1]) * \
             (g - tf.reshape(grad_g / V_norm, [1] * len(norm_axes) + [-1]) * V)

    return V, V_norm, V_scaler, g_param, grad_g, grad_V
# data based initialization for a given Keras model
def add_weightnorm_param_updates(updates, new_V_param, new_g_param, W, V_scaler):
    ps = K.get_variable_shape(new_V_param)
    norm_axes = [i for i in range(len(ps) - 1)]

    # update W and V_scaler
    new_V_norm = tf.sqrt(tf.reduce_sum(tf.square(new_V_param), norm_axes))
    new_V_scaler = new_g_param / new_V_norm
    new_W = tf.reshape(new_V_scaler, [1] * len(norm_axes) + [-1]) * new_V_param
    updates.append(K.update(W, new_W))
    updates.append(K.update(V_scaler, new_V_scaler))

In [119]:
# create model
model = Sequential()
model.add(Dense(180, input_shape=(111,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(180))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
#model.add(BatchNormalization())
model.add(Activation('softmax'))

In [120]:
sgd_wn = SGDWithWeightnorm(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd_wn,metrics=['accuracy'])

In [121]:
model.fit(train_data, train_label,
              batch_size=30,
              epochs=200,
              validation_data=(test_data, test_label),
              shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 15s - loss: 1.0348 - acc: 0.6566 - val_loss: 0.4512 - val_acc: 0.8418
Epoch 2/200
48000/48000 [==============================] - 13s - loss: 0.5934 - acc: 0.7963 - val_loss: 0.4111 - val_acc: 0.8517
Epoch 3/200
48000/48000 [==============================] - 13s - loss: 0.5256 - acc: 0.8158 - val_loss: 0.3862 - val_acc: 0.8590
Epoch 4/200
48000/48000 [==============================] - 15s - loss: 0.4906 - acc: 0.8266 - val_loss: 0.3709 - val_acc: 0.8659
Epoch 5/200
48000/48000 [==============================] - 15s - loss: 0.4676 - acc: 0.8345 - val_loss: 0.3613 - val_acc: 0.8699
Epoch 6/200
48000/48000 [==============================] - 15s - loss: 0.4526 - acc: 0.8390 - val_loss: 0.3545 - val_acc: 0.8696
Epoch 7/200
48000/48000 [==============================] - 16s - loss: 0.4349 - acc: 0.8449 - val_loss: 0.3465 - val_acc: 0.8735
Epoch 8/200
48000/48000 [======================

48000/48000 [==============================] - 13s - loss: 0.2820 - acc: 0.8955 - val_loss: 0.2816 - val_acc: 0.8970
Epoch 64/200
48000/48000 [==============================] - 13s - loss: 0.2830 - acc: 0.8961 - val_loss: 0.2834 - val_acc: 0.8986
Epoch 65/200
48000/48000 [==============================] - 13s - loss: 0.2817 - acc: 0.8954 - val_loss: 0.2827 - val_acc: 0.8974
Epoch 66/200
48000/48000 [==============================] - 13s - loss: 0.2814 - acc: 0.8975 - val_loss: 0.2815 - val_acc: 0.8982
Epoch 67/200
48000/48000 [==============================] - 13s - loss: 0.2788 - acc: 0.8964 - val_loss: 0.2822 - val_acc: 0.8971
Epoch 68/200
48000/48000 [==============================] - 14s - loss: 0.2810 - acc: 0.8958 - val_loss: 0.2820 - val_acc: 0.8967
Epoch 69/200
48000/48000 [==============================] - 15s - loss: 0.2773 - acc: 0.8970 - val_loss: 0.2820 - val_acc: 0.8977
Epoch 70/200
48000/48000 [==============================] - 13s - loss: 0.2799 - acc: 0.8980 - val_loss

KeyboardInterrupt: 